# HW2
Enoch Mwesigwa
Keith Vanderlinden
CS 344
March 17th, 2020

## Exercise 1

In [59]:
from collections import Counter
import numpy as np

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

class SpamFilter:
    def __init__(self,bad_words, good_words):
        #lowercase and flatten lists + get counts of words
        self.good_words = Counter( [ word.lower() for word in np.concatenate(good_words)] )
        self.bad_words = Counter( [ word.lower() for word in np.concatenate(bad_words)] )
        self.construct_prob_table()
        
    # calculates probabilities without as specified by article   
    def calc_probs(self, word):
        g = 2 * self.in_dict(self.good_words, word, .01 )
        b = self.in_dict(self.bad_words, word, .01)
        if (g + b) > 1:
                return ( max(0.01, min(0.99, min(1.0, b / len(self.bad_words)) / 
                                 (min(1.0, g / len(self.good_words)) + min(1.0, b / len(self.bad_words)))))
                       )
        else:
            return 0
    
    #constructs a probability table from the words
    def construct_prob_table(self):
        self.prob_table = dict(map(
        lambda word: (word, self.calc_probs(word)), self.good_words + self.bad_words
    ))
    
    #returns value of if targe exits in dict
    # else returns false value
    def in_dict(self, obj, target, default):
        if target in obj:
            return obj[target] 
        else:
            return default

    #returns the probability of a given word being spam
    def get_prob(self, word):
        return self.in_dict(self.prob_table, word, .5)
    
    def is_spam(self, message):
        probs = list(map(self.get_prob, message))
        product = np.prod(probs)
        product_complement = (product + np.prod( list(map(lambda element: 1 - element, probs)) ))
        return product / product_complement
    
    # calculates and displays the likelihood of multiple messages being spam
    def calc_spam(self, message_list):
        for message in message_list:
            print("\n\n The message", message, "has a ", self.is_spam(message), "chance of being spam")
        
spf = SpamFilter(spam_corpus, ham_corpus)
test_list = ["do i like green eggs".split(), "I do not like spam".split(), "I am green eggs like spam".split()]

spf.calc_spam(test_list)



 The message ['do', 'i', 'like', 'green', 'eggs'] has a  6.407996462188124e-06 chance of being spam


 The message ['I', 'do', 'not', 'like', 'spam'] has a  0.9972779006872371 chance of being spam


 The message ['I', 'am', 'green', 'eggs', 'like', 'spam'] has a  0.25471134298672354 chance of being spam


This approach is Baysian because it uses the Bayes Rule in the sense that we calculate the probability of the cause when we're given it's effect. In this case, the email is the effect. We use it to calculate the probability of whether it is or isn't spam

### Exercise 2

#### A)

In [8]:
from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask

T, F = True, False

weather = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.5}),
    ('Rain', 'Cloudy', {T: 0.8, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0})
])

#### B)

number of random variables = 4
<br/>
number of options for each = 2
<br/>
work: <br/>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2^4 = 16 <br/>
solution:<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   There are 16 independent values in the full joint probability distribution for this domain

#### C)

There are 9 independent values total in the Bayesian network:
* Cloudy: 1 independent value
* Sprinkler: 2 values
* Rain: 2 values
* Wetgrass: 2 values

#### D)

In [15]:
print("i.")
print(enumeration_ask('Cloudy', dict(), weather).show_approx())
'''
    P(Cloudy) = alpha * <P(Cloudy), P(~Cloudy)>

    P(Cloudy) = alpha * <0.5, 0.5>

    P(Cloudy) = <0.5, 0.5>

'''
print()
print("ii.")
print(enumeration_ask('Sprinkler', dict(Cloudy=T), weather).show_approx())

'''
P(Sprinkler | Cloudy) = alpha * <0.1, 0.9>

P(Sprinkler | Cloudy) = <0.1, 0.9>
'''

print()
print("iii.")
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), weather).show_approx())

'''
P(Cloudy | Sprinkler ^ ~Rain) = 
    alpha * [ <P(Cloudy) * P(Sprinkler | Cloudy) * P(~Rain | Cloudy),
    P(~Cloudy) * P(Sprinkler | ~Cloudy) * P(~Rain | ~Cloudy)> ]

P(Cloudy | Sprinkler ^ ~Rain) = alpha * <0.5 * 0.1 * 0.2, 0.5 * 0.5 * 0.8>

P(Cloudy | Sprinkler ^ ~Rain) = <0.952, 0.0476>
'''

print()
print("iv.")
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), weather).show_approx())

'''
    P(WetGrass | Cloudy ^ Sprinkler ^ Rain) = 
    alpha * [ <P(WetGrass | Sprinkler ^ Rain) * P(Sprinkler | Cloudy) * P(Rain | Cloudy) * P(Cloudy),
    P(~WetGrass | Sprinkler ^ Rain) * P(Sprinkler | Cloudy) * P(Rain | Cloudy) * P(Cloudy)> ]

    P(WetGrass | Cloudy ^ Sprinkler ^ Rain) = alpha * <0.99 * 0.1 * 0.8 * 0.5, 0.01 * 0.1 * 0.8 * 0.5>

    P(WetGrass | Cloudy ^ Sprinkler ^ Rain) = <0.99, 0.01>
'''

print()
print("v.")
print(enumeration_ask('Cloudy', dict(WetGrass=F), weather).show_approx())

'''

P(Cloudy | ~WetGrass) = 
    alpha * [
    P(Cloudy, ~WetGrass, Sprinkler, Rain) +
    P(Cloudy, ~WetGrass, Sprinkler, ~Rain) +
    P(Cloudy, ~WetGrass, ~Sprinkler, ~Rain) +
    P(Cloudy, ~WetGrass, ~Sprinkler, Rain)
    ]

P(Cloudy, ~WetGrass, Sprinkler, Rain) = <0.5 * 0.1 * 0.8 * 0.01, 0.5 * 0.5 * 0.2 * 0.01>

P(Cloudy, ~WetGrass, Sprinkler, ~Rain) = <0.5 * 0.1 * 0.2 * 0.1, 0.5 * 0.5 * 0.8 * 0.1>

P(Cloudy, ~WetGrass, ~Sprinkler, ~Rain) = <0.5 * 0.9 * 0.2 * 1, 0.5 * 0.5 * 0.8 1>

P(Cloudy, ~WetGrass, ~Sprinkler, Rain) = <0.5 * 0.9 * 0.8 * 0.1, 0.5 * 0.5 * 0.2 * 0.1>

P(Cloudy | ~WetGrass) = alpha * <0.1274, 0.2255>

P(Cloudy | ~WetGrass) = <0.361, 0.639>

'''
print()

i.
False: 0.5, True: 0.5

ii.
False: 0.9, True: 0.1

iii.
False: 0.952, True: 0.0476

iv.
False: 0.01, True: 0.99

v.
False: 0.639, True: 0.361

